Search satellite by partial satellite name:

Create `space-track-credentials.json` from `space-track-credentials_TEMPLATE.json` first


In [1]:
import requests
import pandas as pd
import os.path
import urllib
import json

In [2]:
# specific satellite name for the rest of this file for convenience
# we search by LIKE so partial match works

# satellite_name = "MICROSAT-R"
# satellite_name = "USA 193"
satellite_name = "FENGYUN 1C"
# satellite_name = "METEOR"
DEBUT_FILE_PATH = 'test/satname-debut-{}.csv'
GP_HISTORY_FILE_PATH = 'test/satname-gp-history-{}.csv'

In [3]:
columns = ["NORAD_CAT_ID","EPOCH","MEAN_MOTION","ECCENTRICITY","INCLINATION","RA_OF_ASC_NODE","ARG_OF_PERICENTER","MEAN_ANOMALY","EPHEMERIS_TYPE","CLASSIFICATION_TYPE","ELEMENT_SET_NO","REV_AT_EPOCH","BSTAR","MEAN_MOTION_DOT","MEAN_MOTION_DDOT","SEMIMAJOR_AXIS","PERIOD","APOAPSIS","PERIAPSIS"]

In [4]:
if "auth_cookie" not in locals():
    auth_cookie = None

In [5]:
auth_cookie

In [6]:
def space_track_login():
    with open("space-track-credentials.json") as json_file:
        credentials = json.load(json_file)
    print("Logging in space-track.org with identity:",credentials['identity'])
    url = "https://www.space-track.org/ajaxauth/login"
    x = requests.post(url, data = credentials)
    if x.status_code == 200:
        print("Logged in")
        return x.cookies
    else:
        print("Failed with code:",x.status_code)
        return False
    
def get_cookie():
    global auth_cookie
    if auth_cookie == None:
        auth_cookie = space_track_login()
    return auth_cookie

In [29]:
def get_satellite_debut(satname):
    csvurl="https://www.space-track.org/basicspacedata/query/class/satcat_debut/SATNAME/~~{}/orderby/INTLDES%20asc/format/csv/metadata/true/".format(satname)
    print("Downloading debut for satname LIKE: ",satname,"\nURL:",csvurl)
    r = requests.get(csvurl, allow_redirects=True, cookies=get_cookie())
    open(DEBUT_FILE_PATH.format(satname), 'wb').write(r.content)
    print("Saved file:",DEBUT_FILE_PATH.format(satname))
    
def get_norad_id_for_satname(satname):
    if os.path.exists(DEBUT_FILE_PATH.format(satname)):
        print("Load from disk:", DEBUT_FILE_PATH.format(satname))
        pass
    else:
        get_satellite_debut(satname)
    df = pd.read_csv(DEBUT_FILE_PATH.format(satname))
    return df.NORAD_CAT_ID.values

def get_satellite_gp_history(norad_ids,satname):
    s = ",".join([str(x) for x in norad_ids])
    csvurl="https://www.space-track.org/basicspacedata/query/class/gp_history/NORAD_CAT_ID/{}/orderby/CCSDS_OMM_VERS asc/format/csv/emptyresult/show".format(s)
    print("Downloading gp history for satname LIKE: ",satname,"\nURL:",csvurl)
    r = requests.get(csvurl, allow_redirects=True, cookies=get_cookie())
    open(GP_HISTORY_FILE_PATH.format(satname), 'wb').write(r.content)
    print("Saved file:",GP_HISTORY_FILE_PATH.format(satname))
    



In [10]:
# get_satellite_gp_history(get_norad_id_for_satname(satellite_name),satellite_name)

In [9]:
s = get_norad_id_for_satname(satellite_name)

Load from disk: test/satname-debut-FENGYUN 1C.csv


In [35]:
# df[columns].to_pickle("test/satname-gp-history-USA 193.pkl.gz","gzip")

In [36]:
# df2 = pd.read_pickle("test/satname-gp-history-USA 193.pkl.gz","gzip")

In [13]:
l = get_norad_id_for_satname(satellite_name)
len(l)
l.columns

Load from disk: test/satname-debut-FENGYUN 1C.csv


AttributeError: 'numpy.ndarray' object has no attribute 'columns'

In [35]:
df = pd.read_csv(DEBUT_FILE_PATH.format(satellite_name))
l2 = df[df.DECAY.notnull()].NORAD_CAT_ID.values
l2

import numpy as np
lll = np.array_split(l2, 20)
lll

[array([30337, 30351, 30353, 30354, 30367, 30376, 30378, 30383, 30384,
        29739, 30399, 30402, 30407, 30410, 30412, 30421, 30422, 30424,
        30430, 30431, 30433, 30440, 30449, 30456, 30457, 30458, 30459,
        30460, 30461, 29742, 30463, 30470, 30482]),
 array([30487, 29743, 30488, 30502, 30511, 29744, 30513, 30517, 30518,
        30534, 30535, 29745, 30536, 30537, 30538, 30544, 30551, 30559,
        30584, 30592, 29747, 30595, 30598, 30601, 30605, 30610, 30614,
        30615, 30617, 30618, 30620, 30626, 30635]),
 array([30638, 29749, 30643, 30644, 30653, 30681, 30682, 30685, 30687,
        30692, 30695, 30698, 30711, 30712, 30713, 29752, 30727, 30729,
        30743, 30747, 30748, 30754, 30757, 30767, 30769, 30801, 30807,
        30808, 30812, 29755, 30818, 30819, 30824]),
 array([30825, 30834, 30839, 30842, 30853, 30854, 30855, 30861, 29757,
        30864, 30865, 30866, 30869, 30881, 30889, 30890, 30894, 30896,
        30897, 30908, 30909, 30910, 30911, 29759, 30917, 30929,

In [37]:
for i,v in enumerate(lll):
    get_satellite_gp_history(v,"FY_DEB_DECAYED_{}".format(i))

URL: https://www.space-track.org/basicspacedata/query/class/gp_history/NORAD_CAT_ID/30337,30351,30353,30354,30367,30376,30378,30383,30384,29739,30399,30402,30407,30410,30412,30421,30422,30424,30430,30431,30433,30440,30449,30456,30457,30458,30459,30460,30461,29742,30463,30470,30482/orderby/CCSDS_OMM_VERS asc/format/csv/emptyresult/show
Saved file: test/satname-gp-history-FY_DEB_DECAYED_0.csv
URL: https://www.space-track.org/basicspacedata/query/class/gp_history/NORAD_CAT_ID/30487,29743,30488,30502,30511,29744,30513,30517,30518,30534,30535,29745,30536,30537,30538,30544,30551,30559,30584,30592,29747,30595,30598,30601,30605,30610,30614,30615,30617,30618,30620,30626,30635/orderby/CCSDS_OMM_VERS asc/format/csv/emptyresult/show
Saved file: test/satname-gp-history-FY_DEB_DECAYED_1.csv
URL: https://www.space-track.org/basicspacedata/query/class/gp_history/NORAD_CAT_ID/30638,29749,30643,30644,30653,30681,30682,30685,30687,30692,30695,30698,30711,30712,30713,29752,30727,30729,30743,30747,30748,30

Saved file: test/satname-gp-history-FY_DEB_DECAYED_18.csv
URL: https://www.space-track.org/basicspacedata/query/class/gp_history/NORAD_CAT_ID/30184,30185,30186,30187,30188,30189,30192,29731,30195,30203,30204,30211,30215,29732,30219,30220,30223,30226,30236,30240,30254,30255,30258,30260,30268,30269,30271,30278,30285,30292,30296,30307/orderby/CCSDS_OMM_VERS asc/format/csv/emptyresult/show
Saved file: test/satname-gp-history-FY_DEB_DECAYED_19.csv


Spacetrack has a really nice API but you still can't download massive amount of data all at once.  They also have a really strict API usage policy.  The above just downloaded DECAYED FY1C stuff.  It's going to be a big undertaking to download all the stuff that's still in orbit.

In [ ]:
# test/satname-gp-history-FY_DEB_DECAYED_19.csv

In [38]:
combined_csv = pd.concat([pd.read_csv("test/satname-gp-history-FY_DEB_DECAYED_{}.csv".format(n)) for n in range(0,20)])


/Users/tim/opt/anaconda3/envs/siads-orbital/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3338: DtypeWarning: Columns (34) have mixed types.Specify dtype option on import or set low_memory=False.
  if (await self.run_code(code, result,  async_=asy)):


In [42]:
combined_csv[columns].to_pickle("test/FY1C_DECAYED_DEB_FULL.pkl.gz","gzip")